In [4]:
!pip install -q kaggle

In [5]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [6]:
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [7]:
!kaggle datasets download -d shanegerami/ai-vs-human-text

Dataset URL: https://www.kaggle.com/datasets/shanegerami/ai-vs-human-text
License(s): other
ai-vs-human-text.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip /content/ai-vs-human-text.zip

Archive:  /content/ai-vs-human-text.zip
replace AI_Human.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import nltk

In [ ]:
nltk.download("all")

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
df =pd.read_csv("/content/AI_Human.csv")
df.head()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
plt.figure(figsize = (10,6))
sns.countplot(x="generated",data =df, palette = ["green","yellow"])
plt.gca().set_xticklabels(["Human","Generated"])
plt.show()

In [ ]:
df["generated"].value_counts()

In [ ]:
df_0 = df[df['generated'] == 0].copy()
df_1 = df[df['generated'] == 1].copy()

In [ ]:
df_0= df_0[:5000]
df_1= df_1[:5000]

In [ ]:
df = pd.concat([df_0,df_1], ignore_index=True)
df.shape

In [ ]:
plt.figure(figsize = (10,6))
sns.countplot(x="generated",data =df, palette = ["green","yellow"])
plt.gca().set_xticklabels(["Human","Generated"])
plt.show()

In [ ]:
df["text"] = df["text"].str.lower()

In [ ]:
df.head()

In [ ]:
nltk.download("punkt")
df["tokens"] = df["text"].apply(word_tokenize)

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
df['tokens'] = df['tokens'].apply(lambda tokens: [word for word in tokens if word not in stop_words])

In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
df['tokens'] = df['tokens'].apply(lambda tokens: [lemmatizer.lemmatize(word) for word in tokens])

In [ ]:
df['preprocessed_text'] = df['tokens'].apply(lambda tokens: ' '.join(tokens))

In [ ]:
df.head()

In [ ]:
selected_columns = ['generated', 'preprocessed_text']
df_Final = df[selected_columns]
print(df_Final.head())

In [ ]:
df_Final.shape

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
df_class_0 = df_Final[df_Final['generated'] == 0]
wordcloud_class_0 = WordCloud(width=800, height=400, background_color='white').generate(' '.join(df_class_0['preprocessed_text']))
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud_class_0, interpolation='bilinear')
plt.title('WordCloud for  Human')
plt.axis('off')
plt.show()
df_class_1 = df_Final[df_Final['generated'] == 1]
wordcloud_class_1 = WordCloud(width=800, height=400, background_color='white').generate(' '.join(df_class_1['preprocessed_text']))
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud_class_1, interpolation='bilinear')
plt.title('WordCloud for Class AI')
plt.axis('off')
plt.show()


In [ ]:
X = df_Final['preprocessed_text']
y = df_Final['generated']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:
from sklearn.svm import SVC
svm_model = SVC()
svm_model.fit(X_train_tfidf, y_train)
svm_predictions = svm_model.predict(X_test_tfidf)

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y_test, svm_predictions))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, svm_predictions))
train_predictions = svm_model.predict(X_train_tfidf)

print("Training Classification Report:\n", classification_report(y_train, train_predictions))
test_predictions = svm_model.predict(X_test_tfidf)
print("\nTesting Classification Report:\n", classification_report(y_test, test_predictions))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
predictions = model.predict(X_test_tfidf)
print("Accuracy:", accuracy_score(y_test, predictions))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, predictions))
train_predictions = model.predict(X_train_tfidf)
print("Training Classification Report:\n", classification_report(y_train, train_predictions))
test_predictions = model.predict(X_test_tfidf)
print("\nTesting Classification Report:\n", classification_report(y_test, test_predictions))

In [ ]:
df_Final.head()

In [ ]:
new_data = ["failure common occurrence one everyone must"]
new_data_tfidf = tfidf_vectorizer.transform(new_data)
predictions = svm_model.predict(new_data_tfidf)
print("SVM Predictions:")
print(predictions)

In [ ]:
new_data = ["america love affair vehicle seems cooling say"]
new_data_tfidf = tfidf_vectorizer.transform(new_data)
predictions = svm_model.predict(new_data_tfidf)
print("SVM Predictions:")
print(predictions)

In [ ]:
new_data = ["failure common occurrence one everyone must"]
new_data_tfidf = tfidf_vectorizer.transform(new_data)
nb_predictions = model.predict(new_data_tfidf)

print("Naive Bayes Predictions:")
print(nb_predictions)

In [ ]:
new_data = ["america love affair vehicle seems cooling say"]
new_data_tfidf = tfidf_vectorizer.transform(new_data)
nb_predictions = model.predict(new_data_tfidf)

print("Naive Bayes Predictions:")
print(nb_predictions)